In [2]:
import os
import struct
import random
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import pydot
import pydotplus
import graphviz
import scipy
import scipy.stats as sts
from scipy.stats import norm
import numpy.random as nrnd
from sklearn import metrics
from keras import regularizers
from keras.layers import Dense, Activation, Input, Dropout, BatchNormalization
from keras.models import Sequential
from keras_tqdm import TQDMNotebookCallback as ktqdm
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.distributions.empirical_distribution import ECDF
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, average_precision_score, confusion_matrix

Using TensorFlow backend.


## Reproducibility

In [3]:
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(42)
tf.set_random_seed(42)

## Reading csv files

In [4]:
path_classes = "/home/tore/venv/homes.di.unimi.it/classes/GM12878.csv"
path_epigenomic = "/home/tore/venv/homes.di.unimi.it/epigenomic-data/GM12878.csv"
path_sequences = "/home/tore/venv/homes.di.unimi.it/sequences/GM12878.csv"

classes = pd.read_csv(path_classes, names = ["Titles"])
epigenomic = pd.read_csv(path_epigenomic, sep=",")

In [11]:
data = (classes.join(epigenomic))
data = data.drop("Unnamed: 0", axis=1)

In [14]:
data = data[(data["Titles"] == "A-P") | (data["Titles"] == "I-P")]

In [22]:
ap = data[data["Titles"] == "A-P"]
ip = data[data["Titles"] == "I-P"]
len(ap), len(ip)

(10816, 73891)

## Creating Training and Testing Sets

    Since I-Ps are seven times more circa than A-Ps, a more fair way to use datas is needed.
    One idea could be separating I-P in seven parts each of 10k elements and running seven batches of training / testing, each one with its own process of k-fold cross evaluation.
    Another one could 